In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
print('Libraries Imported')

Libraries Imported


In [13]:
resp = requests.get("https://en.wikipedia.org/wiki/List_of_largest_banks")
print(resp.text)

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>List of largest banks - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"2ccd6a5a-c95f-412a-b21b-d4b73065b652","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_largest_banks","wgTitle":"List of largest banks","wgCurRevisionId":1036779278,"wgRevisionId":1036779278,"wgArticleId":38298344,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles to be expanded from September 2020","Articles with short description","Short description matches Wikidata","Articles containing poten

In [50]:
soup = BeautifulSoup(resp.content, "html.parser")

In [56]:
table = soup.findAll("table",class_="wikitable sortable mw-collapsible")
table

[<table class="wikitable sortable mw-collapsible">
 <tbody><tr>
 <th data-sort-type="number">Rank
 </th>
 <th>Bank name
 </th>
 <th>Total assets<br/>(2020)<br/>(<a class="mw-redirect" href="/wiki/US$" title="US$">US$</a> Billion)
 </th></tr>
 <tr>
 <td>1
 </td>
 <td><span class="flagicon"><a href="/wiki/China" title="China"><img alt="China" class="thumbborder" data-file-height="600" data-file-width="900" decoding="async" height="15" src="//upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/23px-Flag_of_the_People%27s_Republic_of_China.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/35px-Flag_of_the_People%27s_Republic_of_China.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/45px-Flag_of_the_People%27s_Republic_of_China.svg.png 2x" width="23"/></a></span> <a href="/wiki/Industrial_and_Commercial_Bank_of_China" title=

In [57]:
def removeHTMLTags(html):
    import re
    clean = re.compile("<.*?>")
    text = re.sub(clean,'',html)
    return text

In [74]:
rank_list = []
bank_list = []
asset_list = []
j = 0
for div in table:
    j = j + 1
    if j == 1:
        rows = div.findAll('tr')
        for row in rows:
            tds = row.findAll('td')
            i = 0
            for td in tds:
                td = str(td)
                if i == 0:
                    td = removeHTMLTags(td)
                    td = td.replace("\n","")
                    rank_list.append(td)
                elif i == 1:
                    td = removeHTMLTags(td)
                    td = td.replace("\n","")
                    td = td[1:]
                    bank_list.append(td)
                elif i == 2:
                    td = removeHTMLTags(td)
                    td = td.replace("\n","")
                    td = td.replace(',',"")
                    td = td.replace("[3]","")
                    asset_list.append(removeHTMLTags(td))
                i = i + 1

In [76]:
df = pd.DataFrame({"Rank":rank_list, "Bank":bank_list,"Asset Amt.":asset_list})
df.set_index("Rank", inplace = True)
df.head()

,Bank,Asset Amt.
Rank,,
1,Industrial and Commercial Bank of China,4324.27
2,China Construction Bank,3653.11
3,Agricultural Bank of China,3572.98
4,JPMorgan Chase,3386.07
5,Bank of China,3270.15


In [77]:
df.to_json("Scraped_Bank_Assets.json")

# Crypto Currency API

In [78]:
endpoint = "http://api.coinlayer.com/api/live?access_key=c6742b2379e0c8fed5d381d30c6d73e5&symbols=BTC,ETH"
df = pd.read_json(endpoint)
df.head()

,success,terms,privacy,timestamp,target,rates
BTC,True,https://coinlayer.com/terms,https://coinlayer.com/privacy,2021-08-03 03:12:06,USD,38975.096360
ETH,True,https://coinlayer.com/terms,https://coinlayer.com/privacy,2021-08-03 03:12:06,USD,2558.905877


In [84]:
import datetime
time = str(datetime.datetime.now())
time = time[0:19]
print(time)

2021-08-02 23:16:58


In [86]:
time_list = []
for row in range(df.shape[0]):
    time_list.append(time)
df['Time'] = time_list
df.head()

,success,terms,privacy,timestamp,target,rates,Time
BTC,True,https://coinlayer.com/terms,https://coinlayer.com/privacy,2021-08-03 03:12:06,USD,38975.096360,2021-08-02 23:16:58
ETH,True,https://coinlayer.com/terms,https://coinlayer.com/privacy,2021-08-03 03:12:06,USD,2558.905877,2021-08-02 23:16:58


In [87]:
df.to_csv("Crypto_Exchange.csv")